<a href="https://colab.research.google.com/github/Sangram-1409/AgriSmart/blob/main/Real_time_Cryptocurrency_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The purpose of this script is to fetch and display real-time cryptocurrency prices.
# It uses a public API (CoinGecko) and updates the prices at a regular interval.

# This script can be a foundation for more complex projects, such as:
# 1. Integrating with a carbon token platform to track the value of your earned carbon credits.
# 2. Creating a simple portfolio tracker.
# 3. Building a backend for a web-based price chart.

import requests
import time
import os

# --- Configuration ---
# Set the API endpoint for CoinGecko. This API is free and doesn't require an API key for this use case.
API_URL = "https://api.coingecko.com/api/v3/simple/price"

# Define the cryptocurrencies you want to track and the currencies for the price display.
# The `ids` and `vs_currencies` must match the API's supported list.
# Bitcoin and Ethereum are standard for demonstration.
CRYPTO_IDS = "bitcoin,ethereum"
VS_CURRENCIES = "usd"

# Set the update interval in seconds.
UPDATE_INTERVAL = 10

def clear_screen():
    """Clears the console screen for a clean, real-time display."""
    # os.name is 'nt' for Windows, 'posix' for Linux/macOS
    os.system('cls' if os.name == 'nt' else 'clear')

def fetch_crypto_prices(ids, vs_currencies):
    """
    Fetches cryptocurrency prices from the CoinGecko API.

    Args:
        ids (str): A comma-separated string of cryptocurrency IDs.
        vs_currencies (str): A comma-separated string of price currencies.

    Returns:
        dict: A dictionary of price data or None if the request fails.
    """
    try:
        # Build the request parameters.
        params = {
            "ids": ids,
            "vs_currencies": vs_currencies
        }

        # Make the GET request to the API.
        response = requests.get(API_URL, params=params)

        # Raise an exception for bad status codes (4xx or 5xx).
        response.raise_for_status()

        # Return the JSON data from the response.
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def main():
    """Main function to run the real-time crypto tracker."""
    print("Starting real-time crypto tracker...")
    print("Press Ctrl+C to stop.\n")

    try:
        while True:
            # Fetch the latest prices.
            prices = fetch_crypto_prices(CRYPTO_IDS, VS_CURRENCIES)

            if prices:
                # Clear the screen to display the updated data.
                clear_screen()

                # Print a header with the current time.
                current_time = time.strftime("%Y-%m-%d %H:%M:%S")
                print(f"--- Cryptocurrency Prices ({current_time}) ---")

                # Iterate through the fetched prices and print them.
                for crypto_id, data in prices.items():
                    # Capitalize the cryptocurrency name for display.
                    crypto_name = crypto_id.replace('-', ' ').title()
                    price = data.get(VS_CURRENCIES)

                    if price is not None:
                        # Format the output for readability.
                        print(f"💰 {crypto_name:<10}: ${price:,.2f}")
                    else:
                        print(f"❌ {crypto_name:<10}: Price not available.")

                print("\n-------------------------------------------")

            # Wait for the specified interval before the next update.
            time.sleep(UPDATE_INTERVAL)

    except KeyboardInterrupt:
        # Handle the user pressing Ctrl+C gracefully.
        print("\nTracker stopped by user. Goodbye!")
    except Exception as e:
        # Handle any other unexpected errors.
        print(f"An unexpected error occurred: {e}")

# Ensure the main function runs only when the script is executed directly.
if __name__ == "__main__":
    main()

--- Cryptocurrency Prices (2025-08-20 17:36:28) ---
💰 Bitcoin   : $113,966.00
💰 Ethereum  : $4,307.97

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:36:38) ---
💰 Bitcoin   : $113,966.00
💰 Ethereum  : $4,307.97

-------------------------------------------

Tracker stopped by user. Goodbye!


In [4]:
# This Python script demonstrates the fundamental concepts of a blockchain.
# It is a simplified, single-file implementation to help you understand:
# 1. Blocks: Data containers that are linked together.
# 2. Hashing: The cryptographic function that ensures data integrity.
# 3. Chaining: How each new block points to the previous one, forming a secure chain.

# This is a conceptual tool, not a real cryptocurrency or a decentralized network.

import hashlib
import json
import time

# A Block is the fundamental building block of a blockchain.
# It contains a list of transactions, a timestamp, and a hash of the previous block.
class Block:
    """Represents a single block in a blockchain."""
    def __init__(self, index, transactions, timestamp, previous_hash):
        """
        Initializes a new Block.

        Args:
            index (int): The position of the block in the chain.
            transactions (list): A list of data or transactions to be stored.
            timestamp (float): The time the block was created.
            previous_hash (str): The cryptographic hash of the previous block.
        """
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0  # Nonce for Proof-of-Work (mining)
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """
        Calculates the cryptographic hash of the block's content.
        This hash uniquely identifies the block and ensures its integrity.
        """
        # We need a predictable representation of the block data.
        block_string = json.dumps({
            "index": self.index,
            "transactions": self.transactions,
            "timestamp": self.timestamp,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()

        # Use SHA-256 for hashing, a common cryptographic hash function.
        return hashlib.sha256(block_string).hexdigest()

    def __str__(self):
        """Provides a user-friendly string representation of the Block."""
        return (f"Block #{self.index}\n"
                f"Previous Hash: {self.previous_hash}\n"
                f"Timestamp: {time.ctime(self.timestamp)}\n"
                f"Transactions: {self.transactions}\n"
                f"Nonce: {self.nonce}\n"
                f"Hash: {self.hash}\n")

# The Blockchain is a chain of connected blocks.
# It handles adding new blocks and ensures the chain's integrity.
class Blockchain:
    """Represents the entire blockchain ledger."""
    def __init__(self):
        """Initializes a new blockchain with a genesis (first) block."""
        self.chain = []
        self.pending_transactions = []
        self.create_genesis_block()
        self.difficulty = 4  # The number of leading zeros required for a valid hash

    def create_genesis_block(self):
        """Creates the very first block in the blockchain with no previous hash."""
        genesis_block = Block(0, ["Genesis Block"], time.time(), "0")
        self.chain.append(genesis_block)
        print("Genesis block created.")

    def get_last_block(self):
        """Returns the most recent block in the chain."""
        return self.chain[-1]

    def add_transaction(self, transaction):
        """Adds a new transaction to the list of pending transactions."""
        self.pending_transactions.append(transaction)
        print(f"Transaction added: {transaction}")

    def mine_pending_transactions(self):
        """
        Creates a new block from pending transactions and adds it to the chain.
        This process simulates "mining" by performing a Proof-of-Work.
        """
        if not self.pending_transactions:
            print("No pending transactions to mine.")
            return

        last_block = self.get_last_block()
        new_block = Block(len(self.chain), self.pending_transactions, time.time(), last_block.hash)

        print("Starting to mine the new block...")
        self.proof_of_work(new_block)

        print("\nNew block mined successfully!")
        self.chain.append(new_block)
        self.pending_transactions = []  # Clear the pending transactions
        print(new_block)

    def proof_of_work(self, block):
        """
        A simple Proof-of-Work algorithm to simulate mining.
        The system "works" by finding a hash that meets a specific difficulty (leading zeros).
        """
        target = "0" * self.difficulty
        while not block.hash.startswith(target):
            block.nonce += 1
            block.hash = block.calculate_hash()
            # print(f"Mining... nonce: {block.nonce}, hash: {block.hash}", end='\r')
        print(f"\nProof-of-Work completed. Nonce: {block.nonce}, Hash: {block.hash}")

# --- Main demonstration script ---
if __name__ == "__main__":
    # Create an instance of our simple blockchain.
    my_crypto_ledger = Blockchain()

    # Add some sample transactions.
    my_crypto_ledger.add_transaction({
        "from": "user1",
        "to": "user2",
        "amount": 10
    })

    my_crypto_ledger.add_transaction({
        "from": "user2",
        "to": "user3",
        "amount": 5
    })

    # Mine the first new block.
    my_crypto_ledger.mine_pending_transactions()

    # Add a new transaction after the block has been mined.
    my_crypto_ledger.add_transaction({
        "from": "user3",
        "to": "user1",
        "amount": 3
    })

    # Mine the second new block.
    my_crypto_ledger.mine_pending_transactions()

    print("\n--- Blockchain Status ---")
    for block in my_crypto_ledger.chain:
        print(f"Block #{block.index} Hash: {block.hash}")



Genesis block created.
Transaction added: {'from': 'user1', 'to': 'user2', 'amount': 10}
Transaction added: {'from': 'user2', 'to': 'user3', 'amount': 5}
Starting to mine the new block...

Proof-of-Work completed. Nonce: 65264, Hash: 0000fdb475abb4725282200851a7829d4541c4a6f5d212ba2f964b4dd9e0d61f

New block mined successfully!
Block #1
Previous Hash: cbb569d9cf70a50305befe920e9536b9f269fe4f3ca08702bbe7976a16b0e404
Timestamp: Wed Aug 20 17:37:26 2025
Transactions: [{'from': 'user1', 'to': 'user2', 'amount': 10}, {'from': 'user2', 'to': 'user3', 'amount': 5}]
Nonce: 65264
Hash: 0000fdb475abb4725282200851a7829d4541c4a6f5d212ba2f964b4dd9e0d61f

Transaction added: {'from': 'user3', 'to': 'user1', 'amount': 3}
Starting to mine the new block...

Proof-of-Work completed. Nonce: 55726, Hash: 000013e78ef1e0aa82fd6c9b33b055e7254b33177fda984d09c25551308329ae

New block mined successfully!
Block #2
Previous Hash: 0000fdb475abb4725282200851a7829d4541c4a6f5d212ba2f964b4dd9e0d61f
Timestamp: Wed Aug 2

In [3]:
# This Python script serves as a simple, continuous crypto miner.
# It is designed to work with a simplified blockchain (like the one we built previously).

# The miner's sole job is to find a valid hash for a new block by
# continuously incrementing the 'nonce' until the hash meets a specific difficulty requirement.

# This script can be run in parallel with a blockchain script that generates transactions.

import hashlib
import json
import time
import os
import threading
import sys

# The target difficulty for the mining puzzle.
# A higher number of leading zeros means more difficult mining.
DIFFICULTY = 4

# This class represents the data that a miner would receive from a network.
# In a real network, this data would be broadcast to all miners.
class MiningJob:
    """A data structure representing the information needed to mine a block."""
    def __init__(self, index, transactions, previous_hash):
        self.index = index
        self.transactions = transactions
        self.previous_hash = previous_hash

    def __str__(self):
        """Provides a string representation of the mining job."""
        return (f"Mining Job for Block #{self.index}\n"
                f"Previous Hash: {self.previous_hash[:15]}...\n"
                f"Transactions: {self.transactions}\n")

# A simple miner that continuously tries to find a valid hash.
class Miner:
    """A miner that performs Proof-of-Work to find a valid block hash."""
    def __init__(self, job, difficulty):
        """Initializes the miner with a job and difficulty level."""
        self.job = job
        self.difficulty = difficulty
        self.nonce = 0
        self.hash = ""

    def calculate_hash(self):
        """Calculates the SHA-256 hash for the current job and nonce."""
        block_string = json.dumps({
            "index": self.job.index,
            "transactions": self.job.transactions,
            "timestamp": time.time(), # The timestamp is set at the time of mining
            "previous_hash": self.job.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()

        return hashlib.sha256(block_string).hexdigest()

    def mine(self):
        """
        Performs the Proof-of-Work by trying different nonces.
        Returns the valid hash and the nonce used to find it.
        """
        target = "0" * self.difficulty
        start_time = time.time()

        print("Miner started. Looking for a hash with", self.difficulty, "leading zeros...")

        while not self.hash.startswith(target):
            self.nonce += 1
            self.hash = self.calculate_hash()
            # Provides live feedback on the mining process
            sys.stdout.write(f"\rMining... nonce: {self.nonce:,}, hash: {self.hash[:20]}...")
            sys.stdout.flush()

        end_time = time.time()
        print(f"\n✅ Found valid hash after {self.nonce:,} attempts in {end_time - start_time:.2f} seconds!")
        print(f"Hash: {self.hash}")

        return self.hash, self.nonce

# --- Main demonstration script ---
if __name__ == "__main__":
    # In a real network, the mining job would be received from the blockchain.
    # We will simulate a job here for demonstration purposes.

    # This represents a job to mine Block #1, with a previous hash from a "genesis block"
    # and a set of transactions.
    dummy_transactions = [
        {"from": "Bob", "to": "Alice", "amount": 10},
        {"from": "Charlie", "to": "Bob", "amount": 5}
    ]

    # This is a sample previous hash from a "genesis block"
    previous_hash = "0000216b5a34f4d1c4f5b5f6e8c7c9c0d1b3e8a719d3f4b6a8a3c5d6e7f8g9h0"

    # Create the mining job
    mining_job = MiningJob(index=1, transactions=dummy_transactions, previous_hash=previous_hash)

    # Initialize and run the miner
    miner = Miner(job=mining_job, difficulty=DIFFICULTY)

    try:
        mined_hash, mined_nonce = miner.mine()
        print("\n--- Mining Completed ---")
        print(f"Block Index: {mining_job.index}")
        print(f"Transactions: {mining_job.transactions}")
        print(f"Nonce: {mined_nonce}")
        print(f"Final Hash: {mined_hash}")
        print("--------------------------")
    except KeyboardInterrupt:
        print("\nMiner stopped by user.")


Miner started. Looking for a hash with 4 leading zeros...
Mining... nonce: 55,705, hash: 000037dc7ab975e9ffba...
✅ Found valid hash after 55,705 attempts in 36.86 seconds!
Hash: 000037dc7ab975e9ffba08bc72725faad14b975f365f2445c3118eab686a72dd

--- Mining Completed ---
Block Index: 1
Transactions: [{'from': 'Bob', 'to': 'Alice', 'amount': 10}, {'from': 'Charlie', 'to': 'Bob', 'amount': 5}]
Nonce: 55705
Final Hash: 000037dc7ab975e9ffba08bc72725faad14b975f365f2445c3118eab686a72dd
--------------------------
